# STDP Finds the Start of Repeating Patterns in Continuous Spike Trains

In this notebook we will reproduce the experiments described in [Masquelier & Thorpe (2008)](https://www.semanticscholar.org/paper/Spike-Timing-Dependent-Plasticity-Finds-the-Start-Masquelier-Guyonneau/432b5bfa6fc260289fef45544a43ebcd8892915e).

In [ ]:
# These imports will be used in the notebook
from __future__ import print_function

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

## LIF neuron model

The LIF neuron model used in this experiment is based on Gerstner's [Spike Response Model](http://lcn.epfl.ch/~gerstner/SPNM/node26.html#SECTION02311000000000000000).

At every time-step, the neuron membrane potential p is given by the formula:

$$p=\eta(t-t_{i})\sum_{j|t_{j}>t_{i}}{}w_{j}\varepsilon(t-t_{j})$$

where $\eta(t-t_{i})$ is the membrane response after a spike at time $t_{i}$:

$$\eta(t-t_{i})=K_{1}exp(-\frac{t-t_{i}}{\tau_{m}})-K_{2}(exp(-\frac{t-t_{i}}{\tau_{m}})-exp(-\frac{t-t_{i}}{\tau_{s}}))$$

and $\varepsilon(t)$ describes the Excitatory Post-Synaptic Potential of each synapse spike at time $t_{j}$:

$$\varepsilon(t-t_{j})=K(exp(-\frac{t-t_{j}}{\tau_{m}})-exp(-\frac{t-t_{j}}{\tau_{s}}))$$

Note that K has to be chosen so that the max of $\eta(t)$ is 1, knowing that $\eta(t)$ is maximum when:
$$t=\frac{\tau_{m}\tau_{s}}{\tau_{m}-\tau_{s}}ln(\frac{\tau_{m}}{\tau_{s}})$$

In this simplified version of the neuron, the synaptic weights $w_{j}$ remain constant.

In [ ]:
class LIFNeuron(object):

    def __init__(self,
                 n_syn, W, max_spikes=None, 
                 p_rest=0.0, tau_rest=1.0, tau_m=10.0, tau_s=2.5, T=None,
                 K=2.1, K1=2.0, K2=4.0):

        # Model parameters

        # Membrane resting potential
        self.p_rest = p_rest
        
        # Duration of the recovery period
        self.tau_rest = tau_rest
        
        # Membrane time constant
        self.tau_m = tau_m
        
        # Synaptic time constant
        self.tau_s = tau_s
        
        # Spiking threshold
        if T is None:
            self.T = n_syn/4
        else:
            self.T = T
        
        # Model constants
        self.K = K
        self.K1 = K1
        self.K2 = K2

        # The number of synapses
        self.n_syn = n_syn
        
        # The synapse efficacy weights
        self.w = tf.Variable(W)
        
        # The incoming spike times memory window
        if max_spikes is None:
            self.max_spikes = 70
        else:
            self.max_spikes = max_spikes

        # Placeholders (ie things that are fed to the graph at runtime)

        # A boolean tensor indicating which synapses have spiked during dt
        self.new_spikes = tf.placeholder(shape=[self.n_syn], dtype=tf.bool, name='new_spikes')

        # The time increment since the last update
        self.dt = tf.placeholder(dtype=tf.float32, name='dt')
        
        # Variables (ie things that are modified by the graph at runtime)

        # The neuron memory of incoming spike times
        self.t_spikes = tf.Variable(tf.constant(100000.0, shape=[self.max_spikes, self.n_syn]), dtype=tf.float32)
        
        # The last spike time insertion index
        self.t_spikes_idx = tf.Variable(self.n_syn - 1, dtype=tf.int32)

        # The relative time since the last spike (assume it was a very long time ago)
        self.last_spike = tf.Variable(1000.0, dtype=tf.float32, name='last_spike')
        
        # The membrane potential
        self.p = tf.Variable(self.p_rest,dtype=tf.float32, name='p')
        
        # The duration remaining in the resting period (between 0 and self.tau_s)
        self.t_rest = tf.Variable(0.0,dtype=tf.float32, name='t_rest')

    # Excitatory post-synaptic potential (EPSP)
    def epsilon_op(self):

        # We only use the negative value of the relative spike times
        spikes_t_op = tf.negative(self.t_spikes)

        return self.K *(tf.exp(spikes_t_op/self.tau_m) - tf.exp(spikes_t_op/self.tau_s))
    
    # Membrane spike response
    def eta_op(self):
        
        # We only use the negative value of the relative time
        t_op = tf.negative(self.last_spike)
        
        # Evaluate the spiking positive pulse
        pos_pulse_op = self.K1 * tf.exp(t_op/self.tau_m)
        
        # Evaluate the negative spike after-potential
        neg_after_op = self.K2 * (tf.exp(t_op/self.tau_m) - tf.exp(t_op/self.tau_s))

        # Evaluate the new post synaptic membrane potential
        return self.T * (pos_pulse_op - neg_after_op)
    
    # Neuron behaviour during integrating phase (t_rest = 0)
    def w_epsilons_op(self):
        
        # Evaluate synaptic EPSPs. We ignore synaptic spikes older than the last neuron spike
        epsilons_op = tf.where(tf.logical_and(self.t_spikes >=0, self.t_spikes < self.last_spike),
                               self.epsilon_op(),
                               self.t_spikes*0.0)
                          
        # Agregate weighted incoming EPSPs 
        return tf.reduce_sum(self.w * epsilons_op)

    # Neuron behaviour during resting phase (t_rest > 0)
    def post_firing_p_op(self):
   
        # Membrane potential is only impacted by the last post-synaptic spike (ignore EPSPs)
        return self.eta_op()
    
    def update_spikes_times(self):
        
        # Increase the age of all the existing spikes by dt
        old_spikes_op = self.t_spikes.assign_add(tf.ones(tf.shape(self.t_spikes), dtype=tf.float32) * self.dt)

        # Increment last spike index (modulo max_spikes)
        new_idx_op = self.t_spikes_idx.assign(tf.mod(self.t_spikes_idx + 1, self.max_spikes))

        # Create a list of coordinates to insert the new spikes
        idx_op = tf.constant(1, shape=[self.n_syn], dtype=tf.int32) * new_idx_op
        coord_op = tf.stack([idx_op, tf.range(self.n_syn)], axis=1)

        # Create a vector of new spike times (non-spikes are assigned a very high time)
        new_spikes_op = tf.where(self.new_spikes,
                                 tf.constant(0.0, shape=[self.n_syn]),
                                 tf.constant(100000.0, shape=[self.n_syn]))
        
        # Replace older spikes by new ones
        return tf.scatter_nd_update(old_spikes_op, coord_op, new_spikes_op)
    
    def resting_w_op(self):
        
        # For the base LIF Neuron, the weights remain constants when resting
        return tf.identity(self.w)
    
    def default_w_op(self):
        
        # For the base LIF Neuron, the weights remain constants when integrating
        return tf.identity(self.w)

    def firing_w_op(self):

        # For the base LIF Neuron, the weights remain constants when firing
        return tf.identity(self.w)
    
    def resting_op(self):
        
        # Update weights
        w_op = self.resting_w_op()
        
        # Update the resting period
        t_rest_op = self.t_rest.assign(tf.maximum(self.t_rest - self.dt, 0.0))
        
        # During the resting period, the membrane potential is only given by the eta kernel
        with tf.control_dependencies([w_op, t_rest_op]):
            return self.eta_op()
    
    def firing_op(self):

        # Update weights
        w_op = self.firing_w_op()
        
        # Reset the time of the last spike, but only once the weights have been updated
        with tf.control_dependencies([w_op]):
            last_spike_op = self.last_spike.assign(0.0)

        # Start the resting period
        t_rest_op = self.t_rest.assign(self.tau_rest)
        
        # At spiking time, the membrane potential is only given by the eta kernel
        with tf.control_dependencies([last_spike_op, t_rest_op]):
            return self.eta_op()
        
    def default_op(self):
        
        # Update weights
        w_op = self.default_w_op()
        
        # By default, the membrane potential is given by the sum of the eta kernel and the weighted epsilons
        with tf.control_dependencies([w_op]):
            return self.eta_op() + self.w_epsilons_op()
        
    def integrating_op(self):

        # Evaluate the new membrane potential, integrating both synaptic input and spike dynamics
        p_op = self.eta_op() + self.w_epsilons_op()

        # We have a different behavior if we reached the threshold
        return tf.cond(p_op > self.T,
                       self.firing_op,
                       self.default_op)
    
    def response(self):
        
        # Update our internal memory of the synapse spikes (age older spikes, add new ones)
        update_spikes_op = self.update_spikes_times()
        
        # Increase the relative time of the last spike by the time elapsed
        last_spike_age_op = self.last_spike.assign_add(self.dt)
        
        # Update the internal state of the neuron
        with tf.control_dependencies([update_spikes_op, last_spike_age_op]):
            return tf.cond(self.t_rest > 0.0,
                           self.resting_op,
                           self.integrating_op)

## Stimulate neuron with predefined synapse input

We replicate the figure 3 of the original paper by stimulating a LIF neuron with six consecutive spikes.

The neuron has a refractory period of 1 ms and a threshold of 1.


In [ ]:
# Test neuron response with constant synaptic weights

# Duration of the simulation in ms
T = 80
# Duration of each time step in ms
dt = 1.0
# Number of iterations = T/dt
steps = int(T / dt)
# Number of synapses
n_syn = 1
# Spiking times
spikes = [2.0, 23.0, 44.0, 45.0, 48.0, 61.0, 70.0]
# We define the base synaptic efficacy as a uniform vector
W = np.full((n_syn), 0.475, dtype=np.float32)
# Output variables
P = []

with tf.Session() as sess:

    neuron = LIFNeuron(n_syn, W, T=1)

    sess.run(tf.global_variables_initializer())

    response = neuron.response()
    for step in range(steps):
        
        t = step * dt
        syn_has_spiked = [t in spikes]
        feed = { neuron.new_spikes: syn_has_spiked, neuron.dt: dt}
        p = sess.run(response, feed_dict=feed)
        P.append((t,p))

In [ ]:
# Draw membrane potential
plt.figure()
plt.plot(*zip(*P))
plt.axhline(y=neuron.T, color='r', linestyle='-')
plt.axhline(y=neuron.p_rest, color='y', linestyle='--')
for spike  in spikes:
    plt.axvline(x=spike, color='gray', linestyle='--')
plt.title('LIF response')
plt.ylabel('Membrane Potential (mV)')
plt.xlabel('Time (msec)')

As in the original paper. we see that because of the leaky nature of the neuron, the stimulating spikes have to be nearly synchronous for the threshold to be reached. 

## Generate Poisson spike trains with varying rate

The original paper uses Poisson spike trains with a rate varying in the [0, 90] Hz interval, with a variation speed that itself varies in the [-1800, 1800] Hz interval (in random uniform increments in the [-360,360] interval).

In [ ]:
# A class that generates random spike trains
class SpikeTrains(object):
    
    def __init__(self, n_syn, r_min=0.0, r_max=90.0, r=None, s_max=1800, ds_max=360, s=None, auto_vrate=True, delta_max=0):
        
        # Number of synapses
        self.n_syn = n_syn
        # Minimum and maximum spiking rate (in Hz)
        self.r_min = r_min
        self.r_max = r_max
        # Spiking rate for each synapse (in Hz)
        if r is None:
            self.r = np.random.uniform(self.r_min, self.r_max, size=(n_syn))
        else:
            self.r = r
        # Rate variation parameters
        self.s_max = s_max
        self.ds_max = ds_max
        # Rate variation
        if s is None:
            self.s = np.random.uniform(-self.s_max, self.s_max, size=(self.n_syn))
        else:
            self.s = s
        # Automatically apply rate variation when
        self.auto_vrate = auto_vrate
        # Maximum time between two spikes on each synapse (0 means no maximum) in ms
        self.delta_max = delta_max

        # Memory of spikes
        self.spikes = None
    
    # Generate new spikes for the specified time interval (in ms)
    # The new spikes are added to the existing spike trains.
    # The method returns only the new set of spikes
    def add_spikes(self, t):
        
        for step in range(t):
            # Draw a random number for each synapse
            x = np.random.uniform(0,1, size=(self.n_syn))
            # Each synapse spikes if the drawn number is lower than the probablity
            # given by the integration of the rate over one millisecond
            spikes = x < self.r * 1e-3
            # Keep a memory of our spikes
            if self.spikes is None:
                self.spikes = np.array([spikes])
            else:
                if self.delta_max > 0:
                    # We force each synapse to spike at least every delta_max ms
                    if self.spikes.shape[0] >= self.delta_max - 1:
                        # Get the last delta_max -1 spike trains
                        last_spikes = self.spikes[-(self.delta_max - 1):,:]
                        # For each synapse, count non-zero items
                        n_spikes = np.count_nonzero(last_spikes, axis=0)
                        # Modify spikes to force a spike on synapses where the spike count is zero
                        spikes = np.where(n_spikes > 0, spikes, True)
                # Store spikes
                self.spikes = np.append(self.spikes, [spikes], axis=0)
            if self.auto_vrate:
                self.change_rate()

        return self.spikes[-t:,:]
    
    # Format a list of spike indexes
    def get_spikes(self):
        
        real_spikes = np.argwhere(self.spikes > 0)
        # We prefer having spikes in the range [1..n_syn]
        spike_index = real_spikes[:,1] + 1
        spike_timings = real_spikes[:,0]
        
        return spike_timings, spike_index
    
    # Change rate, applying the specified delta in Hz
    def change_rate(self, delta=None):

        # Update spiking rate
        if delta is None:
            delta = self.s
        self.r = np.clip( self.r + delta, self.r_min, self.r_max)
        # Update spiking rate variation
        ds = np.random.uniform(-self.ds_max, self.ds_max, size=(self.n_syn))
        self.s = np.clip( self.s + ds, -self.s_max, self.s_max)

## Stimulate a LIF Neuron with random spike trains

We now feed the neuron with 500 synapses that generate spikes at random interval with varying rates.

The synaptic efficacy weights are arbitrarily set to 0.475 and remain constant throughout the simulation.

In [ ]:
# Simulation with random spike trains and constant synaptic weights

# Duration of the simulation in ms
T = 3000
# Duration of each time step in ms
dt = 1.0
# Number of iterations = T/dt
steps = int(T / dt)
# Number of synapses
n_syn = 500
# Our random spike trains
spike_trains = SpikeTrains(n_syn)
# Generate spikes over the specified period
syn_has_spiked = spike_trains.add_spikes(T)
# We define the base synaptic efficacy as a uniform vector
W = np.full((n_syn), 0.475, dtype=np.float32)
# Output variables
P = []

with tf.Session() as sess:

    neuron = LIFNeuron(n_syn, W)

    sess.run(tf.global_variables_initializer())

    response = neuron.response()
    for step in range(steps):
        
        t = step * dt
        feed = { neuron.new_spikes: syn_has_spiked[step], neuron.dt: dt}
        p = sess.run(response, feed_dict=feed)
        P.append((t,p))

We draw the neuron membrane response to the 500 random synaptic spike trains.

In [ ]:
plt.rcParams["figure.figsize"] =(15,6)
# Draw input spikes
plt.figure()
plt.axis([0, T, 0, spike_trains.n_syn])
plt.title('Synaptic spikes')
plt.ylabel('synapses')
plt.xlabel('Time (msec)')
t, spikes = spike_trains.get_spikes()
plt.scatter(t, spikes, s=2)
# Draw membrane potential
plt.figure()
plt.plot(*zip(*P))
plt.axhline(y=neuron.T, color='r', linestyle='-')
plt.axhline(y=neuron.p_rest, color='y', linestyle='--')
plt.title('LIF response')
plt.ylabel('Membrane Potential (mV)')
plt.xlabel('Time (msec)')

We can see that the neuron mostly saturates and continuously generates spikes.

## Introduce Spike Timing Dependent Plasticity

We extend the LIFNeuron by allowing it to modify its synapse weights using a Spike Timing Dependent Plasticity algorithm.

The STDP algorithm rewards synapses where spikes occurred immediately before a neuron spike, and inflicts penalties to the synapses where spikes occur after the neuron spike.

The 'rewards' are called Long Term synaptic Potentiation (LTP), and the penalties Long Term synaptic Depression (LTD).

For each synapse that spiked $\Delta{t}$ before a neuron spike:

$$\Delta{w} = a^{+}exp(-\frac{\Delta{t}}{\tau^{+}})$$

For each synapse that spikes $\Delta{t}$ after a neuron spike:

$$\Delta{w} = -a^{-}exp(-\frac{\Delta{t}}{\tau^{-}})$$ 

In [ ]:
class STDPLIFNeuron(LIFNeuron):

    def __init__(self,
                 n_syn, W, max_spikes=None, 
                 p_rest=0.0, tau_rest=1.0, tau_m=10.0, tau_s=2.5, T=None,
                 K=2.1, K1=2.0, K2=4.0,
                 a_plus=0.03125, a_minus=0.0265625, tau_plus=16.8, tau_minus=33.7):
        
        # Call the parent contructor
        super(STDPLIFNeuron, self).__init__(n_syn, W, max_spikes,
                                            p_rest, tau_rest, tau_m, tau_s, T,
                                            K, K1, K2)
        
        self.a_plus = a_plus
        self.tau_plus = tau_plus
        self.a_minus = a_minus
        self.tau_minus = tau_minus
    
    # Long Term synaptic Potentiation
    def LTP_op(self):
        
        # Reward all spikes in our memory that happened before the new spike, but after the previous one
        rewards_op = tf.where(self.t_spikes < self.last_spike,
                              tf.constant(self.a_plus, shape=[self.max_spikes, self.n_syn]) * tf.exp(tf.negative(self.t_spikes/self.tau_plus)),
                              tf.constant(0.0, shape=[self.max_spikes, self.n_syn]))                              
        
        # Accumulate rewards for each synapse along the history axis
        acc_rewards_op = tf.reduce_sum(rewards_op,0)
        
        # Evaluate new weights
        new_w_op = tf.add(self.w, acc_rewards_op)
        
        # Update with new weights clamped to [0,1]
        return self.w.assign(tf.clip_by_value(new_w_op, 0.0, 1.0))
    
    # Long Term synaptic Depression
    def LTD_op(self):

        # Gather all spikes corresponding to the last insertion index
        new_spikes_op = tf.gather(self.t_spikes, self.t_spikes_idx)

        # Inflict penalties, inversely exponential to the time since the last spike
        penalties_op = tf.where(new_spikes_op <= 0.0, # Older spikes at this index have positive times
                                tf.constant(self.a_minus, shape=[self.n_syn]) * tf.exp(tf.negative(self.last_spike/self.tau_minus)),
                                tf.constant(0.0, shape=[self.n_syn]))
        
        # Evaluate new weights
        new_w_op = tf.subtract(self.w, penalties_op)
        
        # Update with new weights clamped to [0,1]
        return self.w.assign(tf.clip_by_value(new_w_op, 0.0, 1.0))

    def firing_w_op(self):
        
        return self.LTP_op()

    def default_w_op(self):
        
        # Apply long-term synaptic depression if we are still close to the last spike
        # Note that if we unconditionally applied the LTD, the weights will slowly
        # decrease to zero if no spike occurs.
        return tf.cond(self.last_spike < self.tau_minus*7,
                       self.LTD_op,
                       lambda: tf.identity(self.w))
    
    def resting_w_op(self):
        
        return self.default_w_op()

## Test STDP with predefined input

We apply the same predefined spike train to an STDP capable LIFNeuron with a limited number of synapses, and draw the resulting rewards (green) and penalties (red).

In [ ]:
# STDP algorithm with predefined input

# Duration of the simulation in ms
T = 80
# Duration of each time step in ms
dt = 1.0
# Number of iterations = T/dt
steps = int(T / dt)
# Number of synapses
m = 5
# Construct an array of spike inputs, initially empty
spikes = np.zeros((steps,m), dtype=np.bool)
# Add a few spikes
spike_times = np.array([2, 23, 44, 45, 48, 61, 70])
spike_index = np.array([1,  3,  2,  0,  4,  1,  3])
spikes[spike_times, spike_index] = True
# We define the base synaptic efficacy as a uniform vector
W = np.full((m), 0.475, dtype=np.float32)
# Output variables
P = []

with tf.Session() as sess:

    neuron = STDPLIFNeuron(m,W)

    sess.run(tf.global_variables_initializer())

    response = neuron.response()
    w_prev = W
    delta_weights = np.zeros((steps, m))
    for step in range(steps):
        
        t = step * dt
        feed = { neuron.new_spikes: spikes[step,:], neuron.dt: dt}
        p = sess.run(response, feed_dict=feed)
        P.append((t,p))
        w_next = neuron.w.eval()
        delta_weights[step,:] = w_next - w_prev
        w_prev = w_next

In [ ]:
plt.rcParams["figure.figsize"] =(6,3)
# Draw input spikes, penalties and rewards
rewards = np.argwhere(delta_weights > 0)
rewards_timings = rewards[:,0]
rewards_index = rewards[:,1] + 1
penalties = np.argwhere(delta_weights < 0)
penalties_timings = penalties[:,0]
penalties_index = penalties[:,1] + 1
plt.figure()
plt.axis([0, T, 0, m + 1])
plt.title('Synaptic spikes')
plt.ylabel('synapses')
plt.xlabel('Time (msec)')
plt.scatter(spike_times, spike_index+1, s=100)
for spike in spike_times:
    plt.axvline(x=spike, color='gray', linestyle='--')
plt.scatter(rewards_timings, rewards_index, color='lightgreen')
plt.scatter(penalties_timings, penalties_index, color='red')
# Draw membrane potential
plt.figure()
for spike in spike_times:
    plt.axvline(x=spike, color='gray', linestyle='--')
plt.axhline(y=neuron.T, color='r', linestyle='-')
plt.axhline(y=neuron.p_rest, color='y', linestyle='--')
plt.plot(*zip(*P))
plt.title('LIF response')
plt.ylabel('Membrane Potential (mV)')
plt.xlabel('Time (msec)')

On the graph above, we verify that the rewards (green dots) are assigned only when the neuron spikes, and that they are assigned to synapses where a spike occured before the neuron spike (big blue dots).

Note: a reward is assigned event if the synapse spike is not synchronous with the neuron spike, but it will be lower.

We also verify that a penaly (red dot) is inflicted on every synapse where a spike occurs after a neuron spike.

Note: these penalties may later be counter-balanced by a reward if a neuron spike closely follows.

## Stimulate an STDP LIF Neuron with random spike trains

The goal here is to check the effects of the STDP learning on the neuron behaviour when it is stimulated with our random spike trains.

In [ ]:
# Stimulate an STDP LIF neuron with random input at varying rates

# Duration of the simulation in ms
T = 3000
# Duration of each time step in ms
dt = 1.0
# Number of iterations = T/dt
steps = int(T / dt)
# Number of synapses
n_syn = 500
# Spike trains
spike_trains = SpikeTrains(n_syn)
# Generate spikes over the specified period
syn_has_spiked = spike_trains.add_spikes(T)
# We define the base synaptic efficacy as a uniform vector
W = np.full((n_syn), 0.475, dtype=np.float32)
# Output variables
P = []

with tf.Session() as sess:

    neuron = STDPLIFNeuron(n_syn, W)

    sess.run(tf.global_variables_initializer())

    response = neuron.response()

    for step in range(steps):
        
        t = step * dt
        feed = { neuron.new_spikes: syn_has_spiked[step], neuron.dt: dt}
        p = sess.run(response, feed_dict=feed)
        P.append((t,p))

In [ ]:
plt.rcParams["figure.figsize"] =(15,6)
# Draw input spikes
spike_timings, spike_index = spike_trains.get_spikes()
plt.figure()
plt.axis([0, T, 0, spike_trains.n_syn])
plt.title('Synaptic spikes')
plt.ylabel('synapses')
plt.xlabel('Time (msec)')
plt.scatter(spike_timings, spike_index, s=1)
# Draw membrane potential
plt.figure()
plt.plot(*zip(*P))
plt.axhline(y=neuron.T, color='r', linestyle='-')
plt.axhline(y=neuron.p_rest, color='y', linestyle='--')
plt.title('LIF response')
plt.ylabel('Membrane Potential (mV)')
plt.xlabel('Time (msec)')

The consequence of this steady stimulation is a low decrease of the synaptic efficacy weights, down to the point where the neuron is not able to fire anymore.

This is an adverse effect of the STDP algorithm used in the original paper.

## Generate recurrent spike trains

We don't follow exactly the same procedure as in the original paper, as the evolution of the hardware and software allows us to generate spike trains more easily. The result, however, is equivalent.

We generate 2000 spike trains, from which we force the 1000 first to repeat a 50 ms pattern at random intervals.

We first define a random 50ms sequence, that will be used as input when the pattern is played.

We then generate random spike trains at every time-step: for the whole population if we are outside the pattern, for half of it otherwise.

The time to the next pattern is chosen with a probability of 0.25 among the next slices of 50 ms (omitting the first one to avoid consecutive patterns).

In [ ]:
# Generate spike trains containing a recurrent pattern

# Duration of the simulation in ms
T = 15000
# Duration of each time step in ms
dt = 1.0
# Number of iterations = T/dt
steps = int(T / dt)
# Number of synapses
n_syn = 2000
# Number of synapses involved in the pattern
n_syn_pattern = int(n_syn/2)
# Output variables
pattern_t = []

# First, instantiate our base spike trains:
# - variable rate,
# - one spike every 50ms at a minimum
spike_trains = SpikeTrains(n_syn, delta_max=50)
spike_trains.add_spikes(T)

# Then instantiate another set of spike trains to represent
# spontaneous activity (10 Hz constant rate)
spike_trains_c = SpikeTrains(n_syn, r=np.full((n_syn), 10), auto_vrate=False)
spike_trains_c.add_spikes(T)

# Create an empty array to host the final spike trains
syn_has_spiked = np.zeros((steps, n_syn), dtype=np.bool)

# We choose the beginning of the pattern in the [25,75] ms interval
pat_start_time = np.random.randint(25,75)
pattern_t.append(pat_start_time)

for step in range(steps):
        
    t = int(step * dt)

    # Initialize our spike vector from our base spike trains
    syn_has_spiked[step,:] = spike_trains.spikes[step,:]

    # Test if we are in the pattern interval
    if t >= pat_start_time and t < (pat_start_time + 50):
        # We just copy the pattern
        syn_has_spiked[step,:n_syn_pattern] = spike_trains.spikes[t - pat_start_time + pattern_t[0],:n_syn_pattern]
    else:
        # Evaluate the time of the next pattern presentation
        if t >= pat_start_time + 100:
            pat_start_time = t
            # We have 1/4 chances of replaying the pattern for each chunk of 50 ms
            r = np.random.uniform(0,1)
            while (r >= 0.25):
                pat_start_time += 50
                r = np.random.uniform(0,1)
            pattern_t.append(pat_start_time)
    # Add spontaneous activity
    syn_has_spiked[step,:] |= spike_trains_c.spikes[step,:]

Display the resulting synapse mean spiking rates, and some samples of the spike trains, identifying the pattern. 

In [ ]:
plt.rcParams["figure.figsize"] =(15,6)
# Evaluate the mean firing rate of each synapse in Hz
rates = np.count_nonzero(syn_has_spiked, axis=0)*1000.0/T
r_max = np.max(rates)
r_mean_a = np.mean(rates[:n_syn_pattern])
r_mean_b = np.mean(rates[n_syn_pattern:])
plt.figure()
plt.title('Synapse mean firing rates')
plt.plot(rates)
plt.axhline(y=r_mean_a, xmax=0.5, color='g', linestyle='--')
plt.text(0,r_max -10,'mean rate: %d' % r_mean_a, color='g')
plt.axhline(y=r_mean_b, xmin=0.5, color='r', linestyle='--')
plt.text(n_syn,r_max -10,'mean rate: %d' % r_mean_b, ha='right', color='r')
intervals = ([0,299],[7200,7499], [14700, 14999])
for interval in intervals:
    it_pattern_t = np.array(pattern_t)
    it_pattern_t = it_pattern_t[np.logical_and(it_pattern_t >=interval[0], it_pattern_t <=interval[1])]
    # Draw input spikes, identifying the patterns
    plt.figure()
    it_spikes = syn_has_spiked[interval[0]:interval[1]]
    it_real_spikes = np.argwhere(it_spikes)
    for pat_t in it_pattern_t:
        plt.fill_between((pat_t,pat_t+50,pat_t+50,pat_t),(0,0,n_syn_pattern,n_syn_pattern),facecolor='lightgray')
    t, s = it_real_spikes.T
    plt.scatter(interval[0] + t,s+1,s=1)

We verify that the mean spiking rate is the same for both population of synapses (64 Hz = 54 Hz + 10 Hz).

We nevertheless notice that the standard deviation is much higher for the synapses involved in the pattern. 

On the spike trains samples, one can identify the patterns, slightly modified by the 10 Hz spontaneous activity.

## Stimulate an STDP LIF neuron with recurrent spiking trains

We perform a simulation on our STDP LIF neuron with the generated spike trains.

In [ ]:
# Simulation with recurrent pattern

# We define the base synaptic efficacy as a uniform vector
W = np.full((n_syn), 0.475, dtype=np.float32)
# Output variables
P = []

with tf.Session() as sess:

    neuron = STDPLIFNeuron(n_syn, W)

    sess.run(tf.global_variables_initializer())

    response = neuron.response()
    
    for step in range(steps):
        
        t = int(step * dt)

        # Evaluate the neuron response
        feed = { neuron.new_spikes: syn_has_spiked[step], neuron.dt: dt}
        p = sess.run(response, feed_dict=feed)
        P.append((t,p))

In [ ]:
plt.rcParams["figure.figsize"] =(15,3)
intervals = ([0,1999],[7000,8999], [13000, 14999])
for interval in intervals:
    it_P = P[interval[0]:interval[1]]
    it_pattern_t = np.array(pattern_t)
    it_pattern_t = it_pattern_t[np.logical_and(it_pattern_t >=interval[0], it_pattern_t <=interval[1])]
    # Draw membrane potential, identifying the patterns
    plt.figure()
    for pat_t in it_pattern_t:
        plt.fill_between((pat_t,pat_t+50,pat_t+50,pat_t),(-neuron.T/2,-neuron.T/2,neuron.T*2,neuron.T*2),facecolor='lightgray')
    plt.plot(*zip(*it_P))
    plt.axhline(y=neuron.T, color='r', linestyle='-')
    plt.axhline(y=neuron.p_rest, color='y', linestyle='--')
    plt.title('LIF response')
    plt.ylabel('Membrane Potential (mV)')
    plt.xlabel('Time (msec)')

To our disappointment, the neuron quickly saturates and doesn't learn anything !!